**Sample ID**: 348

**Query**:

Update the 'Software Licenses' spend category name to 'Software Subscriptions' in workday.

**DB Type**: Base Case

**Case Description**:

The finance data steward intends to rename an existing Workday spend category. The category currently named as Software Licenses needs to be updated to Software Subscriptions to reflect broader usage.

```
<multiturn info>
Query 2: Now, send a message in channel. (Follow Up Request)
Slack Channel: finance-updates (Information Gathering)  
Slack Message: "Spend category 'Software Licenses' has been updated to 'Software Subscriptions' in Workday." (Information Gathering)  
</multiturn info>
```

**Global/Context Variables:**


**APIs:**

- workday
- slack

# Set Up

## Download relevant files

In [1]:
import io
import os
import sys
import zipfile
import shutil
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Version to download
VERSION = "0.1.4"  # This will be replaced dynamically

# Define paths
CONTENT_DIR = '/content'
APIS_DIR = os.path.join(CONTENT_DIR, 'APIs')
DBS_DIR = os.path.join(CONTENT_DIR, 'DBs')
SCRIPTS_DIR = os.path.join(CONTENT_DIR, 'Scripts')
FC_DIR = os.path.join(CONTENT_DIR, 'Schemas')
ZIP_PATH = os.path.join(CONTENT_DIR, f'APIs_V{VERSION}.zip')

# Google Drive Folder ID where versioned APIs zip files are stored
APIS_FOLDER_ID = '1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4'

# List of items to extract from the zip file
ITEMS_TO_EXTRACT = ['APIs/', 'DBs/', 'Scripts/']

# Clean up existing directories and files
for path in [APIS_DIR, DBS_DIR, SCRIPTS_DIR, FC_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)

# Authenticate and create the drive service
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Helper function to download a file from Google Drive
def download_drive_file(service, file_id, output_path, file_name=None, show_progress=True):
    """Downloads a file from Google Drive"""
    destination = output_path
    request = service.files().get_media(fileId=file_id)
    with io.FileIO(destination, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if show_progress:
                print(f"Download progress: {int(status.progress() * 100)}%")

# 1. List files in the specified APIs folder
print(f"Searching for APIs zip file with version {VERSION} in folder: {APIS_FOLDER_ID}...")
apis_file_id = None

try:
    query = f"'{APIS_FOLDER_ID}' in parents and trashed=false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])
    for file in files:
        file_name = file.get('name', '')
        if file_name.lower() == f'apis_v{VERSION.lower()}.zip':
            apis_file_id = file.get('id')
            print(f"Found matching file: {file_name} (ID: {apis_file_id})")
            break

except Exception as e:
    print(f"An error occurred while listing files in Google Drive: {e}")

if not apis_file_id:
    print(f"Error: Could not find APIs zip file with version {VERSION} in the specified folder.")
    sys.exit("Required APIs zip file not found.")

# 2. Download the found APIs zip file
print(f"Downloading APIs zip file with ID: {apis_file_id}...")
download_drive_file(drive_service, apis_file_id, ZIP_PATH, file_name=f'APIs_V{VERSION}.zip')

# 3. Extract specific items from the zip file to /content
print(f"Extracting specific items from {ZIP_PATH} to {CONTENT_DIR}...")
try:
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_contents = zip_ref.namelist()

        for member in zip_contents:
            extracted = False
            for item_prefix in ITEMS_TO_EXTRACT:
                if member == item_prefix or member.startswith(item_prefix):
                    zip_ref.extract(member, CONTENT_DIR)
                    extracted = True
                    break

except zipfile.BadZipFile:
    print(f"Error: The downloaded file at {ZIP_PATH} is not a valid zip file.")
    sys.exit("Invalid zip file downloaded.")
except Exception as e:
    print(f"An error occurred during extraction: {e}")
    sys.exit("Extraction failed.")

# 4. Clean up
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

# 5. Add APIs to path
if os.path.exists(APIS_DIR):
    sys.path.append(APIS_DIR)
else:
    print(f"Error: APIS directory not found at {APIS_DIR} after extraction. Cannot add to path.")

# 6. Quick verification
# Check for the presence of the extracted items
verification_paths = [APIS_DIR, DBS_DIR, SCRIPTS_DIR]
all_present = True
print("\nVerifying extracted items:")
for path in verification_paths:
    if os.path.exists(path):
        print(f"✅ {path} is present.")
    else:
        print(f"❌ {path} is MISSING!")
        all_present = False

if all_present:
    print(f"\n✅ Setup complete! Required items extracted to {CONTENT_DIR}.")
else:
    print("\n❌ Setup failed! Not all required items were extracted.")

# 7. Generate Schemas
from Scripts.FCSpec import generate_package_schema

print("\nGenerating FC Schemas")
os.makedirs(FC_DIR, exist_ok=True)

# Change working directory to the source folder
os.chdir(APIS_DIR)

# Iterate through the packages in the /content/APIs directory
for package_name in os.listdir(APIS_DIR):
    package_path = os.path.join(APIS_DIR, package_name)

    # Check if it's a directory (to avoid processing files)
    if os.path.isdir(package_path):
        # Call the function to generate schema for the current package
        generate_package_schema(package_path, output_folder_path=FC_DIR)
print(f"✅ Successfully generated {len(os.listdir(FC_DIR))} FC Schemas to {FC_DIR}")
os.chdir(CONTENT_DIR)

Searching for APIs zip file with version 0.1.4 in folder: 1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4...
Found matching file: APIs_V0.1.4.zip (ID: 1TnAaWGfVrMxWTilyhy46-Aue_bh0XkNk)
Download progress: 100%
Extracting specific items from /content/APIs_V0.1.4.zip to /content...

Verifying extracted items:
✅ /content/APIs is present.
✅ /content/DBs is present.
✅ /content/Scripts is present.

✅ Setup complete! Required items extracted to /content.

Generating FC Schemas
-> Processing package: spotify
  - ERROR: An unexpected error occurred for 'get_current_user_profile' (spotify.user_profile.get_current_user_profile): No module named 'jsonpath_ng'
  - ERROR: An unexpected error occurred for 'get_user_top_artists_and_tracks' (spotify.user_profile.get_user_top_artists_and_tracks): No module named 'jsonpath_ng'
  - ERROR: An unexpected error occurred for 'get_user_profile' (spotify.user_profile.get_user_profile): No module named 'jsonpath_ng'
  - ERROR: An unexpected error occurred for 'follow_playlist

/content/APIs/whatsapp/SimulationEngine/models.py:26: SyntaxWarning: invalid escape sequence '\d'
  PHONE_NUMBER = "^\d+$"  # Matches strings containing only digits


  - ERROR: An unexpected error occurred for 'create_refund' (stripe.refund.create_refund): No module named 'jsonpath_ng'
  - ERROR: An unexpected error occurred for 'list_payment_intents' (stripe.payment.list_payment_intents): No module named 'jsonpath_ng'
  - ERROR: An unexpected error occurred for 'list_subscriptions' (stripe.subscription.list_subscriptions): No module named 'jsonpath_ng'
  - ERROR: An unexpected error occurred for 'cancel_subscription' (stripe.subscription.cancel_subscription): No module named 'jsonpath_ng'
  - ERROR: An unexpected error occurred for 'update_subscription' (stripe.subscription.update_subscription): No module named 'jsonpath_ng'
  - ERROR: An unexpected error occurred for 'list_coupons' (stripe.coupon.list_coupons): No module named 'jsonpath_ng'
  - ERROR: An unexpected error occurred for 'create_coupon' (stripe.coupon.create_coupon): No module named 'jsonpath_ng'
  - ERROR: An unexpected error occurred for 'update_dispute' (stripe.dispute.update_disp

## Install Dependencies and Clone Repositories

In [ ]:
!pip install -r /content/APIs/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 91.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 112.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.1/245.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.3/187.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.0 MB/s eta 0:00:

## Import APIs and initiate DBs

In [ ]:
# proto_ignore
import random
import sys
import uuid
import secrets

# Import libraries to ensure all initializations by the python libaries are complete
import workday
import slack

def patch_randomness(seed=42):
    rng = random.Random(seed)
    random.seed(seed)

    # Patch uuid.uuid4
    def deterministic_uuid4():
        return uuid.UUID(int=rng.getrandbits(128))
    sys.modules['uuid'].uuid4 = deterministic_uuid4

    # Patch secrets to use the same deterministic random generator
    class DeterministicRandom:
        def randbelow(self, n):
            return rng.randrange(n)

        def choice(self, seq):
            return rng.choice(seq)

        def randbits(self, k):
            return rng.getrandbits(k)

        def randint(self, a, b):
            return rng.randint(a, b)
    sys.modules['secrets'] = DeterministicRandom()

patch_randomness()

In [ ]:
import workday
import slack
import datetime

# Load initial DB state
print("Loading initial DB state...")
workday.SimulationEngine.db.load_state("/content/DBs/WorkdayStrategicSourcingDefaultDB.json")
slack.SimulationEngine.db.load_state("/content/DBs/SlackDefaultDB.json")
print("Initial DB state loaded.")

# Workday: Create Spend Categories
print("Setting up Workday spend categories...")
spend_categories_to_create = [
    {'name': 'Software Licenses', 'external_id': 'software_licenses', 'usages': ['procurement']},
    {'name': 'Hardware', 'external_id': 'hardware_equipment', 'usages': ['procurement']},
    {'name': 'Office Supplies', 'external_id': 'office_supplies', 'usages': ['procurement']},
    {'name': 'Consulting Services', 'external_id': 'consulting_svcs', 'usages': ['procurement']},
    {'name': 'Travel Expenses', 'external_id': 'travel_exp', 'usages': ['expense']},
    {'name': 'Marketing Budget', 'external_id': 'mktg_budget', 'usages': ['procurement']},
    {'name': 'Employee Benefits', 'external_id': 'emp_benefits', 'usages': ['expense']}
]

for category_data in spend_categories_to_create:
    try:
        created_category = workday.create_spend_category(
            name=category_data.get('name', ''),
            external_id=category_data.get('external_id', ''),
            usages=category_data.get('usages', [])
        )
        category_name = created_category.get('name')
        print(f"Created Workday spend category: {category_name}")
    except Exception as e:
        print(f"Error creating spend category {category_data.get('name')}: {e}")

# Slack: Create User and Channel
print("\nSetting up Slack resources...")
try:
    # Create a user for Slack operations
    test_user = slack.invite_admin_user(email="rovman.paul@gmail.com", real_name="rovman paul")
    test_user_id = test_user.get('user', {}).get('id')
    if test_user_id:
        print(f"Created Slack user: {test_user.get('user', {}).get('real_name')} with ID: {test_user_id}")
    else:
        print(f"Failed to create Slack user: {test_user.get('error')}")

    # Create the #finance-updates channel
    finance_channel = slack.create_channel(name="finance-updates")
    if finance_channel.get('ok'):
        print(f"Created Slack channel: #{finance_channel.get('channel', {}).get('name')}")
    else:
        print(f"Error creating Slack channel #finance-updates: {finance_channel.get('error')}")

except Exception as e:
    print(f"An error occurred during Slack setup: {e}")

print("\nDB setup completed.")

Loading initial DB state...
Initial DB state loaded.
Setting up Workday spend categories...
Created Workday spend category: None
Created Workday spend category: None
Created Workday spend category: None
Created Workday spend category: None
Created Workday spend category: None
Created Workday spend category: None
Created Workday spend category: None

Setting up Slack resources...
Created Slack user: rovman paul with ID: U5A0CEF64
Created Slack channel: #finance-updates

DB setup completed.


# Initial Assertion

1. Assert that a spend category with name 'Software Licenses' exists in the Workday system.
2. Assert that no spend category already exists with the name 'Software Subscriptions'.
3. Assert that the Slack channel "finance-updates" exists and is accessible for posting updates.
4. Assert that no Slack message has been posted in finance-updates.


In [ ]:
import workday
import slack
from Scripts.assertions_utils import compare_strings, compare_is_string_subset

# --- Constants ---
#
SPEND_CATEGORY_NAME = "Software Licenses"
NON_EXISTENT_SPEND_CATEGORY_NAME = "Software Subscriptions"
SLACK_CHANNEL_NAME = "finance-updates"

# --- Pre-compute / Data Gathering ---

# 1) Workday: fetch spend category by external_id
spend_category_error = None
spend_category_exists = False
spend_category_details = {}
SPEND_CATEGORY_EXTERNAL_ID = None
try:
    all_cat =  workday.list_spend_categories()
    for x in all_cat['data']:
        if x.get("name") == SPEND_CATEGORY_NAME:
            SPEND_CATEGORY_EXTERNAL_ID = x.get( "external_id")
            break


    spend_category_details = workday.get_spend_category_by_id(external_id=SPEND_CATEGORY_EXTERNAL_ID) or {}
    if spend_category_details.get("external_id") == SPEND_CATEGORY_EXTERNAL_ID:
        spend_category_exists = True
except Exception as e:
    spend_category_error = str(e)

# 2) Workday: ensure NO category already exists with the target (new) name
list_categories_error = None
category_with_new_name_found = False
categories_seen_sample = []
try:
    all_spend_categories = workday.list_spend_categories() or []
    for idx, category in enumerate(all_spend_categories['data']):
        # Access the name directly from the dictionary in the list
        name = category.get("name", "")
        if idx < 5:
            categories_seen_sample.append(name)
        if compare_strings(name, NON_EXISTENT_SPEND_CATEGORY_NAME):
            category_with_new_name_found = True
            break
except Exception as e:
    list_categories_error = str(e)

# 3) Slack: find #finance-updates channel
channels_error = None
finance_channel_found = False
finance_channel_id = None
channels_list_sample = []
try:
    channels_response = slack.list_channels(types="public_channel,private_channel") or {}
    if channels_response.get("ok"):
        for ch in channels_response.get("channels", []) or []:
            ch_name = ch.get("name", "")
            if len(channels_list_sample) < 5:
                channels_list_sample.append(ch_name)
            if compare_strings(ch_name, SLACK_CHANNEL_NAME):
                finance_channel_found = True
                finance_channel_id = ch.get("id")
                break
    else:
        channels_error = channels_response.get("error") or "unknown_error"
except Exception as e:
    channels_error = str(e)

# 4) Slack: verify NO recent message contains the expected update substring
history_error = None
no_message_found = False
if finance_channel_id and (channels_error is None):
    try:
        history_response = slack.get_conversation_history(channel=finance_channel_id) or {}
        if history_response.get("ok"):
            messages = history_response.get("messages", [])
            if len(messages) == 0:
                no_message_found = True
        else:
            history_error = history_response.get("error") or "unknown_error"
    except Exception as e:
        history_error = str(e)

# --- Assertions ---

# Assertion 1: Spend category with given external_id exists
assertion_condition_1 = (spend_category_error is None) and spend_category_exists
assertion_message_1 = (
    f"Assertion 1 Failed: Spend category with external_id '{SPEND_CATEGORY_EXTERNAL_ID}' should exist. "
)
if spend_category_error:
    assertion_message_1 += f"Workday API error: {spend_category_error}."
else:
    assertion_message_1 += f"Observed keys: {list(spend_category_details.keys())[:6]}."
assert assertion_condition_1, assertion_message_1

# Assertion 2: No spend category already exists with the name 'Software Subscriptions'
assertion_condition_2 = (list_categories_error is None) and (not category_with_new_name_found)
assertion_message_2 = (
    f"Assertion 2 Failed: A spend category with the name '{NON_EXISTENT_SPEND_CATEGORY_NAME}' was found."
)
if list_categories_error:
    assertion_message_2 += f" Workday list categories error: {list_categories_error}."
else:
    assertion_message_2 += f" Sample names: {categories_seen_sample}."
assert assertion_condition_2, assertion_message_2

# Assertion 3: Slack channel '#finance-updates' exists
assertion_condition_3 = (channels_error is None) and finance_channel_found and (finance_channel_id is not None)
assertion_message_3 = f"Assertion 3 Failed: Slack channel '{SLACK_CHANNEL_NAME}' should exist."
if channels_error:
    assertion_message_3 += f" Slack channels API error: {channels_error}."
else:
    assertion_message_3 += f" Sample channels: {channels_list_sample}."
assert assertion_condition_3, assertion_message_3

# Assertion 4: No recent Slack message about the update has been posted
assertion_condition_4 = (history_error is None) and (no_message_found)
assertion_message_4 = (
    f"Assertion 4 Failed: An update message about '{SPEND_CATEGORY_EXTERNAL_ID}' was found in the channel."
)
if history_error:
    assertion_message_4 += f" Slack history API error: {history_error}."
else:
    assertion_message_4 += f" Sample messages: {messages[:5]}."
assert assertion_condition_4, assertion_message_4

# Action

In [ ]:
# proto_ignore

import workday
import slack

**Simulated User**: Update the 'Software Licenses' spend category name to 'Software Subscriptions'.

**Action Agent**: I need to update a spend category name. To do this, I'll first need to get the spend category's information to ensure I'm updating the correct one. What is the external ID of the spend category you would like to update?

**Simulated User**: I don't have the external ID, but the current name of the spend category is 'Software Licenses'.

In [ ]:
workday.list_spend_categories()

{'data': [{'id': 'SC001',
   'name': 'Office Supplies',
   'external_id': 'EXT001',
   'usages': ['Procurement', 'Expense Tracking']},
  {'id': 'SC002',
   'name': 'IT Equipment',
   'external_id': 'EXT002',
   'usages': ['Hardware Purchase', 'Software Licensing']},
  {'id': 'SC003',
   'name': 'Marketing',
   'external_id': 'EXT003',
   'usages': ['Advertising', 'Promotions', 'Sponsorships']},
  {'id': 'SC004',
   'name': 'Travel & Lodging',
   'external_id': 'EXT004',
   'usages': ['Business Travel', 'Conferences', 'Hotel Bookings']},
  {'id': 'SC005',
   'name': 'Professional Services',
   'external_id': 'EXT005',
   'usages': ['Consulting', 'Legal', 'Freelance Work']},
  {'id': 'SC006',
   'name': 'Software Licenses',
   'external_id': 'software_licenses',
   'usages': ['procurement']},
  {'id': 'SC007',
   'name': 'Hardware',
   'external_id': 'hardware_equipment',
   'usages': ['procurement']},
  {'id': 'SC008',
   'name': 'Office Supplies',
   'external_id': 'office_supplies',
 

In [ ]:
workday.update_spend_category_by_id(external_id='software_licenses', name='Software Subscriptions')

{'data': {'type': 'spend_categories',
  'id': 'SC006',
  'attributes': {'name': 'Software Subscriptions',
   'external_id': 'software_licenses',
   'usages': ['procurement']}}}

**Action Agent**: I have successfully updated the spend category 'Software Licenses' to 'Software Subscriptions'. Would you like me to notify anyone about this change?

**Simulated User**: Yes, I'd like to send a notification about this change.

**Action Agent**: Of course. Who should I notify, and what should the message say? Do you have a preference for email or a Slack message?

**Simulated User**: Please send a Slack message to the finance-updates channel. The message should say: "Spend category 'Software Licenses' has been updated to 'Software Subscriptions' in Workday."

In [ ]:
slack.list_channels(types='public_channel,private_channel')

{'ok': True,
 'channels': [{'messages': [{'ts': '1688682784.334459',
     'user': 'U04L7NE5Q1Y',
     'text': "Welcome everyone to the marketing brainstorming session!  Let's kick off by sharing any initial campaign ideas for Q3.",
     'reactions': [{'name': 'rocket',
       'users': ['U04L7NE5Q1Y', 'U04M2R8JCQ6', 'U04M526DV51'],
       'count': 3}]},
    {'ts': '1688683000.456789',
     'user': 'U04M2R8JCQ6',
     'text': 'I think we should focus on a social media campaign highlighting our sustainability initiatives.',
     'reactions': [{'name': 'thumbsup',
       'users': ['U04L7NE5Q1Y', 'U04M526DV51', 'U04LMCYSD2X'],
       'count': 3}]},
    {'ts': '1688684000.987654',
     'user': 'U04LMCYSD2X',
     'text': 'Has anyone seen those interactive ads on platform X?',
     'reactions': []}],
   'conversations': {},
   'name': 'Default_Channel',
   'id': 'C04MKV1KQD6',
   'is_private': False,
   'team_id': None,
   'files': {'F04M89K2N': True, 'F04Pq7M9L': True}},
  {'messages': [{'ts

In [ ]:
slack.post_chat_message(text="Spend category 'Software Licenses' has been updated to 'Software Subscriptions' in Workday.", channel='CEE29502E')

{'ok': True,
 'message': {'channel': 'CEE29502E',
  'text': "Spend category 'Software Licenses' has been updated to 'Software Subscriptions' in Workday.",
  'attachments': None,
  'blocks': None,
  'user': 'bot',
  'ts': '1759503697.0358562',
  'as_user': None,
  'icon_emoji': None,
  'icon_url': None,
  'link_names': None,
  'markdown_text': None,
  'metadata': None,
  'mrkdwn': None,
  'parse': None,
  'reply_broadcast': None,
  'thread_ts': None,
  'unfurl_links': None,
  'unfurl_media': None,
  'username': None}}

**Action Agent**: I have posted the update to the finance-updates channel. Is there anything else I can help you with?

# Final Assertion

1. Assert that the spend category with name  'Software Licenses' has been successfully renamed to 'Software Subscriptions' in Workday.
2. Assert that a message has been posted in the "finance-updates" slack channel.


In [ ]:
import workday
import slack
from Scripts.assertions_utils import *
import datetime

# Define constants for assertions
SPEND_CATEGORY_NAME = 'Software Subscriptions'
EXPECTED_SPEND_CATEGORY_NAME = 'Software Subscriptions'

# Assertion 1: Assert that the spend category has been renamed.
category_renamed = False
workday_api_error = None
SPEND_CATEGORY_EXTERNAL_ID = None
try:
    all_cat =  workday.list_spend_categories()
    for x in all_cat['data']:
        if x.get("name") == SPEND_CATEGORY_NAME:
            SPEND_CATEGORY_EXTERNAL_ID = x.get( "external_id")
            break


    spend_category_details = workday.get_spend_category_by_id(external_id=SPEND_CATEGORY_EXTERNAL_ID)
    if spend_category_details:
        current_name = spend_category_details.get('name', '')
        if compare_strings(current_name, EXPECTED_SPEND_CATEGORY_NAME):
            category_renamed = True
except Exception as e:
    workday_api_error = str(e)

assertion_message_1 = ("Assertion 1 Failed:"
+ f"workday_api_error {workday_api_error}" if workday_api_error else
                       f"The spend category '{SPEND_CATEGORY_EXTERNAL_ID}' was not renamed to '{EXPECTED_SPEND_CATEGORY_NAME}'."
                       )

assert category_renamed, assertion_message_1


# --- Assertion 2: A message has been posted in the "finance-updates" slack channel." ---
api_error_history = None
channel_name = 'finance-updates'
one_message_found = False

try:
  #Find channel id by name
  channels = slack.list_channels(types="public_channel").get("channels", [])
  channel_id = next(
      (c["id"] for c in channels if compare_strings(c.get("name", ""), channel_name)), None)

  #Get conversation history
  messages = slack.get_conversation_history(channel=channel_id).get("messages", [])
  if len(messages) == 1:
    one_message_found = True
except Exception as e:
    api_error_history = str(e)

cond2 = api_error_history is None and one_message_found
msg2 = f"Assertion 2 Failed: Expected exactly 1 message in {channel_name}. API error: {api_error_history}."
assert cond2, msg2
